In [53]:
import face_recognition
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [54]:
df = pd.read_csv('annotation.csv')

In [55]:
df

,image,height,weight,BMI
0,f_001,1.55,61.0,25.390219
1,f_002,1.76,85.0,27.440599
2,f_003,1.78,56.0,17.674536
3,f_004,1.63,63.0,23.711845
4,f_005,1.76,54.0,17.432851
...,...,...,...,...
1021,m_509,1.91,116.0,31.797374
1022,m_510,1.93,111.0,29.799458
1023,m_511,1.88,109.0,30.839746
1024,m_512,1.78,75.0,23.671254


In [56]:
dff = df[0:513]
dfm = df[513:]

In [57]:
def get_face_encoding(image_path):
    try:
        picture_of_me = face_recognition.load_image_file(image_path)
    except FileNotFoundError:
        print(image_path + " no file found")
        return np.zeros(128).tolist()
    my_face_encoding = face_recognition.face_encodings(picture_of_me)
    if not my_face_encoding:
        print(image_path + " no face found !!!")
        return np.zeros(128).tolist()
    return my_face_encoding[0].tolist()

In [58]:
all_faces = []
for image_name in df.image:
  face_enc = get_face_encoding("D:\\Projects\\Machine Learning\\BMI\\vip\\vip\\data\\data\\"+image_name+".jpg")
  all_faces.append(face_enc)

D:\Projects\Machine Learning\BMI\vip\vip\data\data\f_090.jpg no face found !!!
D:\Projects\Machine Learning\BMI\vip\vip\data\data\f_325.jpg no face found !!!
D:\Projects\Machine Learning\BMI\vip\vip\data\data\f_406.jpg no face found !!!
D:\Projects\Machine Learning\BMI\vip\vip\data\data\m_118.jpg no face found !!!
D:\Projects\Machine Learning\BMI\vip\vip\data\data\m_130.jpg no face found !!!
D:\Projects\Machine Learning\BMI\vip\vip\data\data\m_196.jpg no face found !!!
D:\Projects\Machine Learning\BMI\vip\vip\data\data\m_373.jpg no face found !!!
D:\Projects\Machine Learning\BMI\vip\vip\data\data\m_390.jpg no face found !!!
D:\Projects\Machine Learning\BMI\vip\vip\data\data\m_452.jpg no face found !!!
D:\Projects\Machine Learning\BMI\vip\vip\data\data\m_455.jpg no face found !!!
D:\Projects\Machine Learning\BMI\vip\vip\data\data\m_493.jpg no face found !!!


In [59]:
X = np.array(all_faces) ## This is the training data matrix
y_height = df.height.values ## all labels
y_weight = df.weight.values
y_BMI = df.BMI.values
plotX = pd.DataFrame(X)

In [60]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_height_train, y_height_test, y_weight_train, y_weight_test ,y_BMI_train, y_BMI_test = train_test_split(X, y_height,y_weight,y_BMI, random_state=69)

In [61]:
def report_goodness(model, X_test, y_test, predictor_log=True):
    # Make predictions using the testing set
    y_pred = model.predict(X_test)
    y_true = y_test

    if predictor_log:
        y_true = np.log(y_test)
    # The coefficients
    # The mean squared error
    print("Mean squared error: %.2f"      % mean_squared_error(y_true, y_pred))
    # Explained variance score: 1 is perfect prediction
    print('Variance score: %.2f' % r2_score(y_true, y_pred))
    
    errors = abs(y_pred - y_true)
    mape = 100 * np.mean(errors / y_true)
    accuracy = 100 - mape
    print('Model Performance')
    print('Average Error: {:0.4f} degrees.'.format(np.mean(errors)))
    print('Accuracy = {:0.2f}%.'.format(accuracy))

In [62]:
from sklearn.kernel_ridge import KernelRidge
from sklearn import  linear_model
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

In [63]:
model_BMI = linear_model.LinearRegression()
model_BMI = model_BMI.fit(X_train, np.log(y_BMI_train))
report_goodness(model_height, X_test, y_BMI_test)

Mean squared error: 6.57
Variance score: -236.34
Model Performance
Average Error: 2.5588 degrees.
Accuracy = 18.01%.


In [64]:
model_BMI = Ridge(fit_intercept=True, alpha=0.0015, random_state=4, normalize=True)
model_BMI = model_BMI.fit(X_train, np.log(y_BMI_train))
report_goodness(model_BMI, X_test, y_BMI_test)

Mean squared error: 0.02
Variance score: 0.31
Model Performance
Average Error: 0.0922 degrees.
Accuracy = 97.11%.


In [65]:
model_BMI = RandomForestRegressor(max_depth=2, random_state=0, n_estimators=100)
model_BMI = model_BMI.fit(X_train, np.log(y_BMI_train))
report_goodness(model_BMI, X_test, y_BMI_test)

Mean squared error: 0.02
Variance score: 0.22
Model Performance
Average Error: 0.0979 degrees.
Accuracy = 96.92%.


In [52]:
model_BMI_kridge = KernelRidge(kernel='rbf', gamma=0.21, alpha=0.0017)
model_BMI_kridge = model_BMI_kridge.fit(X_train, np.log(y_BMI_train))
report_goodness(model_BMI_kridge, X_test, y_BMI_test)

Mean squared error: 0.02
Variance score: 0.28
Model Performance
Average Error: 0.0986 degrees.
Accuracy = 96.91%.


In [84]:
from sklearn.svm import SVR
model_SVR = SVR()
model_SVR = model_SVR.fit(X_train, np.log(y_BMI_train))
report_goodness(model_SVR, X_test, y_BMI_test)

Mean squared error: 0.02
Variance score: 0.34
Model Performance
Average Error: 0.0904 degrees.
Accuracy = 97.17%.


In [71]:
model_SVRh = SVR(kernel = 'linear')
model_SVRh = model_SVRh.fit(X_train, np.log(y_BMI_train))
report_goodness(model_SVRh, X_test, y_BMI_test)

Mean squared error: 0.02
Variance score: 0.30
Model Performance
Average Error: 0.0927 degrees.
Accuracy = 97.10%.


In [74]:
from sklearn.linear_model import Lasso
model_Lasso = Lasso(alpha=0.00)
model_Lasso = model_Lasso.fit(X_train, np.log(y_BMI_train))
report_goodness(model_Lasso, X_test, y_BMI_test)

Mean squared error: 0.02
Variance score: 0.31
Model Performance
Average Error: 0.0923 degrees.
Accuracy = 97.11%.


C:\Users\osaru\.conda\envs\CompVis\lib\site-packages\ipykernel_launcher.py:3: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\osaru\.conda\envs\CompVis\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:532: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  positive)
C:\Users\osaru\.conda\envs\CompVis\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:532: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 4.653325703425932, tolerance: 0.002020208245859056
  positive)


In [81]:
df['weight'].to_frame().head(15)

,weight
0,61.0
1,85.0
2,56.0
3,63.0
4,54.0
5,56.0
6,54.0
7,57.0
8,63.0
9,51.0


In [82]:
srk = []

In [88]:
srk = get_face_encoding("D:\\Projects\\Machine Learning\\srk.jpg")

In [93]:
test_array_srk = np.expand_dims(np.array(get_face_encoding("D:\\Projects\\Machine Learning\\srk.jpg")), axis=0)

In [94]:
bmis = np.exp(model_SVR.predict(test_array_srk)).item()

In [95]:
bmi

25.856873813311804

In [96]:
test_array_k = np.expand_dims(np.array(get_face_encoding("D:\\Projects\\Machine Learning\\kohli.jpg")), axis=0)

In [97]:
bmik = np.exp(model_SVR.predict(test_array_k)).item()

In [98]:
bmik

25.75980971133189

In [99]:
test_array_sh = np.expand_dims(np.array(get_face_encoding("D:\\Projects\\Machine Learning\\shiv.jpg")), axis=0)

In [100]:
bmish = np.exp(model_SVR.predict(test_array_sh)).item()

In [101]:
bmish

30.82502944343218